In [6]:
from glob import glob

import pandas as pd
import re

import sys,os,argparse


NAN = float('nan')
INPUT_PATH = "data/1/{}.csv"
OUTPUT_PATH = "data/2/{}.csv"

Notes on status definitions:
* http://www.espn.com/nfl/story/_/id/17361111/nfl-streamlines-injured-players-game-status-removes-probable-designation
* http://harvardsportsanalysis.org/2013/11/inaccuracies-in-the-injury-report-across-the-nfl/

I will be coding non-injured to 0 and all other statuses to 1. "Out" players should not be appearing in the data.

# Import data

In [7]:
def injury_df_from_files(files):
    df = pd.concat([pd.read_csv(file) for file in files])
    df['name'] = df['name'].apply(reformat_name)
    df['status_code'] = [get_status_code(status) for status in df['status']]
    df.sort_values(['name', 'season', 'week'], inplace=True)
    if 'Unnamed: 0' in df.columns:
        df.drop(columns='Unnamed: 0', inplace=True)
    return df

# Reformat name, convert injury to status code

In [12]:
def reformat_name(name):
    return ' '.join(name.split(', ')[::-1])

def get_status_code(status):
    injury_status = ['questionable', 'doubtful', 'day-to-day', 'out']
    status = status.lower()
    if status in injury_status:
        return 1
    return 0

# ... now run everything

In [11]:
INJURY_FILES = glob(INPUT_PATH.format("*injury*"))
assert len(INJURY_FILES) > 0, "where are my files???"

injury_df = injury_df_from_files(INJURY_FILES)